# Gráficos  - Projeto 3 - QTA

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Análise histórico ponto de coleta ponte socorro


In [28]:
relatorio_qualidade_2005_2010 = pd.read_excel("./planilhas/RelatorioQualidadeAguasSuperficiais-2005-2010.xlsx", engine='openpyxl')
relatorio_qualidade_2010_2015 = pd.read_excel("./planilhas/RelatorioQualidadeAguasSuperficiais-2010-2015.xlsx", engine='openpyxl')
relatorio_qualidade_2015_2020 = pd.read_excel("./planilhas/RelatorioQualidadeAguasSuperficiais-2015-2020.xlsx", engine='openpyxl')
relatorio_qualidade_2020_2021 = pd.read_excel("./planilhas/RelatorioQualidadeAguasSuperficiais-2020-2021.xlsx", engine='openpyxl')
all_dfs = [relatorio_qualidade_2005_2010, relatorio_qualidade_2010_2015, relatorio_qualidade_2015_2020, relatorio_qualidade_2020_2021]
historico = pd.concat(all_dfs)

In [54]:
def get_data_from_parameter(parameter, multiple_parameters = False):
    if not multiple_parameters:
        df = historico[historico["Parametro"] == parameter][["Data Coleta", "Valor"]]
    else:
        df = historico[historico["Parametro"].isin(parameter)][["Parametro", "Data Coleta", "Valor"]]
    
    return df


ph = get_data_from_parameter("pH")
temp_agua = get_data_from_parameter("Temperatura da Água")
temp_ar = get_data_from_parameter("Temperatura do Ar")
coliformes = get_data_from_parameter("Escherichia coli**")
solidos_suspensos = get_data_from_parameter("Sólido Suspenso Total")
turbidez = get_data_from_parameter("Turbidez")
nitrogenio = get_data_from_parameter(["Nitrogênio Kjeldahl", "Nitrogênio-Nitrito", "Nitrogênio-Nitrato", "Nitrogênio Amoniacal"], multiple_parameters=True)
ox_dissolvido = get_data_from_parameter("Oxigênio Dissolvido")
dbo = get_data_from_parameter("DBO (5, 20)")




In [57]:
nitrogenio

,Parametro,Data Coleta,Valor
149,Nitrogênio Kjeldahl,28/01/2009,"3,51000000"
150,Nitrogênio Kjeldahl,18/03/2009,"9,46000000"
151,Nitrogênio Kjeldahl,27/05/2009,"22,80000000"
152,Nitrogênio Kjeldahl,17/06/2009,"20,90000000"
153,Nitrogênio Kjeldahl,15/07/2009,"15,60000000"
...,...,...,...
114,Nitrogênio Amoniacal,19/02/2020,"8,91000000"
115,Nitrogênio Amoniacal,14/07/2020,"11,10000000"
116,Nitrogênio Amoniacal,03/09/2020,"8,40000000"
117,Nitrogênio Amoniacal,05/11/2020,"11,00000000"


### Dados Coletados 

In [56]:
pH = 0.12
DBO = 0.10
OD = 0.17
temp = 0.10
nitro = 0.10
turb = 0.08
fos = 0.1
res = 0.08
fosf = 0.1
coli = 0.15
list_data = [DBO, nitro, coli, pH, OD, temp, fosf, turb, res]